In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
        
from scipy.stats import skew,norm,zscore
from scipy.signal import periodogram

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
sns.set_theme()

# Importing the dataset

In [ ]:
orig_holidays_events = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv", parse_dates=['date'])
orig_oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv", parse_dates=['date'])
orig_stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
orig_transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv", parse_dates=['date'])

In [ ]:
orig_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv", parse_dates=['date'])
orig_train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=['date'])

In [ ]:
fig = plt.figure(figsize=(18,7))
orig_train.groupby(by='date')['sales'].mean().plot()

In [ ]:
date = {}

In [ ]:
date['date_start_train'] = '2013-01-01'
date['date_end_train'] = '2017-08-15'
date['date_start_test'] = '2017-08-16'
date['date_end_test'] = '2017-08-31'
date['date_start_fore'] = '2016-06-01'

In [ ]:

diff_train = (pd.Timestamp(date['date_end_train']) - pd.Timestamp(date['date_start_fore'])).days
diff_test = (pd.Timestamp(date['date_end_test']) - pd.Timestamp(date['date_start_fore'])).days

In [ ]:
orig_stores.sample(10)

In [ ]:
fig, axes = plt.subplots(54, 1, figsize=(15, 54*3))

for ax, i in zip(axes.flat, np.arange(0,54,1)):
    sns.lineplot(ax=axes[i], x=orig_train[orig_train.store_nbr==(i+1)].groupby(by='date')['sales'].mean().reset_index().date.values, y=orig_train[orig_train.store_nbr==(i+1)].groupby(by='date')['sales'].mean())
    ax.set_title(f'store_n_{i}') 
    
fig.tight_layout()

In [ ]:
orig_stores.groupby(by=['city']).store_nbr.nunique().sort_values(ascending=False)

In [ ]:
def store_func (orig_df):
    
    df = orig_df.copy()
    
    # Adding features to orig_stores
    df['uniquestore'] = df.city.apply(lambda x: 0 if x in ['Quito', 'Guayaquil', 'Santo Domingo', 'Cuenca', 'Manta', 'Machala', 'Latacunga', 'Ambato'] else 1)
    df['newstore'] = df.store_nbr.apply(lambda x: 1 if x in [19, 20, 21, 28, 35, 41, 51, 52] else 0)
        
    # Merging orig_stores, orig_test and orig_train
    df = pd.concat([orig_train, orig_test], axis=0).merge(df, on=['store_nbr'], how='left')
    df = df.rename(columns={'type' : 'store'}) 

    return df

In [ ]:
final_df = store_func(orig_stores)

In [ ]:
orig_holidays_events.sample(5)

In [ ]:
orig_holidays_events.query("type=='Event'").description.value_counts()

In [ ]:
orig_holidays_events[orig_holidays_events[['date', 'locale_name']].duplicated()]

In [ ]:
orig_holidays_events.query("date in ('2012-12-24', '2012-12-31', '2014-12-26', '2016-05-01', '2016-05-07', '2016-05-08', '2016-07-24')")

In [ ]:
orig_holidays_events.description[orig_holidays_events.description.str.match(r'.*ster.*')]

In [ ]:
orig_train.groupby(by=['store_nbr','date'])['sales'].mean().reset_index().head()

In [ ]:
orig_train.groupby(by=['store_nbr','date'])['sales'].mean().reset_index().query("date.dt.month==1 and date.dt.day==1").sample(10)

In [ ]:
orig_holidays_events.query("transferred==True")

In [ ]:
fig = plt.figure(figsize=(10,5))
orig_train.query("(date.dt.year==2017 and date.dt.month==1) or (date.dt.year==2016 and date.dt.month==12)").groupby(by='date')['sales'].mean().plot()
plt.axvline(x=pd.Timestamp('2017-01-01'), color='black', linestyle='--', linewidth=1, alpha=0.5)

In [ ]:
orig_holidays_events.query("date.dt.month==1 and date.dt.day==1")

In [ ]:
def holiday_func (orig_df):
    
    df = orig_df.copy()
    
    # Non-transferred events
    df.loc[297, 'transferred'] = df.loc[297, 'transferred'] = False
    df = df.query("transferred!=True")
    
    # Removing duplicates
    df = df.drop(index=orig_holidays_events[orig_holidays_events[['date', 'locale_name']].duplicated()].index.values)

    # Adding event type
    df.loc[df.type=='Event', 'type'] = df.description.apply(lambda x: x[0:7])
     
    # Merging orig_holidays_events and final_df
    nat_df = df.query("locale=='National'")
    loc_df = df.query("locale=='Local'")
    reg_df = df.query("locale=='Regional'")
    
    df = final_df.merge(nat_df, left_on=['date'], right_on=['date'], how='left')
    df = df.merge(loc_df, left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')
    df = df.merge(reg_df, left_on=['date', 'state'], right_on=['date', 'locale_name'], how='left')
   
    # Adding New Year
    df['firstday'] = df.description_x.apply(lambda x: 1 if x=='Primer dia del ano' else 0)

    # Matching event and store
    df = df.drop(columns=['locale_x', 'locale_name_x', 'description_x', 'transferred_x',
                          'locale_y', 'locale_name_y', 'description_y', 'transferred_y',
                          'locale', 'locale_name', 'description', 'transferred'])
    df.loc[~df.type_x.isnull(), 'event_type'] = df.type_x.apply(lambda x: x)
    df.loc[~df.type_y.isnull(), 'event_type'] = df.type_y.apply(lambda x: x)
    df.loc[~df.type.isnull(), 'event_type'] = df.type.apply(lambda x: x)
    df.loc[df.event_type.isnull(), 'event_type'] = df.event_type.apply(lambda x: 'norm')
    df = df.drop(columns=['type_x', 'type_y', 'type'])

    df['isevent'] = df.event_type.apply(lambda x: 'y' if x!='norm' else 'n')

    # Adding Easter
    df.loc[df.date.isin(['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']), 'isevent'] = df.isevent.apply(lambda x: 'y')
    df.loc[df.date.isin(['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']), 'event_type'] = df.event_type.apply(lambda x: 'Holiday')

    # Adding closure days
    df['isclosed'] = df.groupby(by=['date', 'store_nbr'])['sales'].transform(lambda x: 1 if x.sum()==0 else 0)    
    df.loc[(df.date.dt.year==2017) & (df.date.dt.month==8) & (df.date.dt.day>=16) , 'isclosed'] = df.isclosed.apply(lambda x: 0)    
    df.loc[df.date.isin(['2017-01-01']), 'isevent'] = df.isevent.apply(lambda x: 'n')
  
    return df

In [ ]:
final_df = holiday_func(orig_holidays_events)

In [ ]:
sns.lineplot(y=orig_oil.dcoilwtico, x=orig_oil.date)

In [ ]:
orig_oil.set_index('date').resample("D").mean().isnull().sum()

In [ ]:
def oil_func (orig_df):
    
    df = orig_df.copy()
    
    # Adding missing values
    df = df.set_index('date').resample("D").mean().interpolate(limit_direction='backward').reset_index()
    
    # Adding new features
    df['lagoil_1_dcoilwtico'] = df['dcoilwtico'].shift(1)
    df['lagoil_2_dcoilwtico'] = df['dcoilwtico'].shift(2)
    df['lagoil_3_dcoilwtico'] = df['dcoilwtico'].shift(3)
    df['lagoil_4_dcoilwtico'] = df['dcoilwtico'].shift(4)
    df['oil_week_avg'] = df['dcoilwtico'].rolling(7).mean()

    df.dropna(inplace = True)
    
    # Merging orig_oil and final_df
    df = final_df.merge(df, on=['date'], how='left')
    
    return df

In [ ]:
final_df = oil_func(orig_oil)

In [ ]:
orig_transactions.sample(5)

In [ ]:
def transactions_func (orig_df):
    
    df = orig_df.copy()
    
    # Merging orig_transactions and final_df
    df = final_df.merge(df, on=['date', 'store_nbr'], how='left')
    
    # Filling missing values
    df.loc[(df.transactions.isnull()) & (df.isclosed==1), 'transactions'] = df.transactions.apply(lambda x: 0)
    group_df = df.groupby(by=['store_nbr', 'date']).transactions.first().reset_index()
    group_df['avg_tra'] = group_df.transactions.rolling(15, min_periods=10).mean()
    group_df.drop(columns='transactions', inplace=True)
    df = df.merge(group_df, on=['date', 'store_nbr'], how='left')
    df.loc[(df.transactions.isnull()) & (df.isclosed==0), 'transactions'] = df.avg_tra
    df.drop(columns='avg_tra', inplace=True)
    df.loc[(df.date.dt.year==2017) & (df.date.dt.month==8) & (df.date.dt.day>=16) , 'transactions'] = df.transactions.apply(lambda x: None)    

    df['tot_store_day_onprom'] = df.groupby(by=['date', 'store_nbr']).onpromotion.transform(lambda x: x.sum())

    return df

In [ ]:
final_df = transactions_func(orig_transactions)

In [ ]:
fig = plt.figure(figsize=(18,7))
final_df.groupby(by='date')['transactions'].mean().plot()

In [ ]:
del orig_train
del orig_test
del orig_stores
del orig_holidays_events
del orig_oil
del orig_transactions

In [ ]:
final_df.sample(5)

In [ ]:
final_df = final_df.set_index('date').loc[date['date_start_fore']:,:]

In [ ]:
final_df.isnull().sum()

In [ ]:
final_df.groupby(by=['city']).sales.sum().sort_values(ascending=False)

In [ ]:
final_df.query("city=='Quito'").groupby(by=['store_nbr']).sales.sum().sort_values(ascending=False)

In [ ]:
final_df.groupby(by='store').cluster.nunique()

In [ ]:
final_df.groupby(by=['store']).sales.sum().sort_values(ascending=False)

In [ ]:
final_df.groupby(by=['cluster']).sales.sum().sort_values(ascending=False)

In [ ]:
final_df.groupby(by=['family']).sales.sum().sort_values(ascending=False)

In [ ]:
def plot_periodogram(ts, detrend='linear', ax=None):
    fs = pd.Timedelta("365D") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(ts, fs=fs, detrend=detrend, window="boxcar", scaling='spectrum')
    
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(["Annual (1)", "Semiannual (2)", "Quarterly (4)", 
                        "Bimonthly (6)", "Monthly (12)", "Biweekly (26)", 
                        "Weekly (52)", "Semiweekly (104)"], rotation=30)
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    
    return ax

In [ ]:
_ = plot_periodogram(final_df.loc['2015-01-01':'2017-08-15'].groupby(by='date').sales.mean())

In [ ]:
_ = plot_periodogram(final_df.query("family=='GROCERY I'").loc['2015-01-01':'2017-08-15'].groupby(by='date').sales.mean())

In [ ]:
_ = plot_periodogram(final_df.query("family=='BEVERAGES'").loc['2015-01-01':'2017-08-15'].groupby(by='date').sales.mean())

In [ ]:
def split_func (orig_df, X, y, end_date, test_size):
    
    # Splitting train and test
    idx_train, idx_test = train_test_split(orig_df.index, test_size=test_size, shuffle=False)
    X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
    y_train, y_test = y.loc[idx_train], y.loc[idx_test]
    
    return X_train, y_train, X_test, y_test

In [ ]:
def add_features (orig_df):
    
    df = orig_df.copy()
        
    # Time features
    df['year'] = df.index.year.astype('int')
    df['quarter'] = df.index.quarter.astype('int')
    df['month'] = df.index.month.astype('int')
    df['day'] = df.index.day.astype('int')
    df['dayofweek'] = df.index.day_of_week.astype('int')
    df['weekofyear'] = df.index.week.astype('int')
    df['isweekend'] = df.dayofweek.apply(lambda x: 1 if x in (5,6) else 0)
    df['startschool'] = df.month.apply(lambda x: 1 if x in (4,5,8,9) else 0)
    
    df['daysinmonth'] = df.index.days_in_month.astype('int')

    # Dummy features
    df = pd.get_dummies(df, columns=['year'], drop_first=True)
    df = pd.get_dummies(df, columns=['quarter'], drop_first=True)
    df = pd.get_dummies(df, columns=['dayofweek'], drop_first=True)
    df = pd.get_dummies(df, columns=['store'], drop_first=True)
    df = pd.get_dummies(df, columns=['event_type'], drop_first=True)
    df = pd.get_dummies(df, columns=['isevent'], drop_first=True)
    df = pd.get_dummies(df, columns=['state'], drop_first=True)

    # DeterministicProcess
    fourierA = CalendarFourier(freq='A', order=5)
    fourierM = CalendarFourier(freq='M', order=2)
    fourierW = CalendarFourier(freq='W', order=4)

    dp = DeterministicProcess(index=df.index,
                          order=1,
                          seasonal=False,
                          constant=False,
                          additional_terms=[fourierA, fourierM, fourierW],
                          drop=True)
    dp_df = dp.in_sample()
    df = pd.concat([df, dp_df], axis=1)
    
    # Outliers
    df['outliers'] = df.sales.apply(lambda x: 1 if x>30000 else 0)
    
    df.drop(columns=['daysinmonth', 'month', 'city'], inplace=True)
    
    return df

In [ ]:
def model_func (orig_df, end_df, n):
    
    df = add_features(orig_df).loc[:end_df,:].reset_index().set_index(['store_nbr', 'family', 'date']).sort_index()
    y = np.log1p(df.loc[:,'sales'].unstack(['store_nbr', 'family']))
    
    # Selecting features
    cols = df.columns[df.columns.str.match(r'year|quarter|event|isevent|isclosed|oil\_week|dcoilwtico|week|isweek|startschool|sin|cos|trend')]
    X = df.loc[:,cols]
    X = X.groupby(by='date').first()
    
    X_tr, y_tr, X_te, y_te = split_func(y, X, y, end_df, n)
    
    # Train
    if end_df <= date['date_end_train']:
        y_tr = np.empty((diff_train-n,0))
        y_te = np.empty((n,0))
        pred_train_y = np.empty((diff_train-n,0))
        pred_test_y = np.empty((n,0))
    # Test
    else:
        y_tr = np.empty((diff_test-n,0))
        y_te = np.empty((n,0))
        pred_train_y = np.empty((diff_test-n,0))
        pred_test_y = np.empty((n,0))
    
    # A model for each shop
    for i in orig_df.store_nbr.unique():
        y = df.loc[i,'sales'].unstack(['family'])
        X = df.loc[i,cols]
        X = X.groupby(by='date').first()

        # Splitting train and test and log transformation
        X_train, y_train, X_test, y_test = split_func(y, X, np.log1p(y), end_df, n)
        
        # Linear regression
        model = LinearRegression()
        model.fit(X_train, y_train)
        lr_pred_train_y = model.predict(X_train) 
        lr_pred_test_y = model.predict(X_test)
        
        # Random Forest
        model = RandomForestRegressor(n_estimators=320, random_state=0)
        model.fit(X_train, y_train)
        rf_pred_train_y = model.predict(X_train) 
        rf_pred_test_y = model.predict(X_test)
        
        # Average result
        st_pred_train_y = lr_pred_train_y * 0.5 + rf_pred_train_y * 0.5
        st_pred_test_y = lr_pred_test_y * 0.5 + rf_pred_test_y * 0.5
        
        y_tr = np.append(y_tr, y_train, axis=1)
        y_te = np.append(y_te, y_test, axis=1)
        pred_train_y = np.append(pred_train_y, st_pred_train_y, axis=1)
        pred_test_y = np.append(pred_test_y, st_pred_test_y, axis=1)
        
        # Performances of each shop
        # Train
        if end_df <= date['date_end_train']:
            print(f'RMSLE_train st_n {i}: ', np.round(np.sqrt(mean_squared_error(y_train.clip(0.0), st_pred_train_y.clip(0.0))), 4), f'RMSLE_test st_n {i}: ', np.round(np.sqrt(mean_squared_error(y_test.clip(0.0), st_pred_test_y.clip(0.0))), 4))
        # Test
        else:
            print(f'RMSLE_train st_n {i}: ', np.round(np.sqrt(mean_squared_error(y_train.clip(0.0), st_pred_train_y.clip(0.0))), 4))
        
    iterables = [final_df.store_nbr.unique(), final_df.family.sort_values().unique()]
    index = pd.MultiIndex.from_product(iterables, names=['store_nbr', 'family'])
    
    y_tr = pd.DataFrame(y_tr, columns=index, index=X_tr.index)
    y_te = pd.DataFrame(y_te, columns=index, index=X_te.index)
    pred_train_y = pd.DataFrame(pred_train_y, columns=y_tr.columns, index=y_tr.index)
    pred_test_y = pd.DataFrame(pred_test_y, columns=y_te.columns, index=y_te.index)
    
    # Total performances
    # Train
    if end_df <= date['date_end_train']:
        print(f'RMSLE_train tot: ', np.round(np.sqrt(mean_squared_error(y_tr.clip(0.0), pred_train_y.clip(0.0))), 4), f'RMSLE_test tot: ', np.round(np.sqrt(mean_squared_error(y_te.clip(0.0), pred_test_y.clip(0.0))), 4))
    # Test
    else:
        print(f'RMSLE_train tot: ', np.round(np.sqrt(mean_squared_error(y_tr.clip(0.0), pred_train_y.clip(0.0))), 4)) 
   
    y_tr = y_tr.stack(['store_nbr', 'family'])
    y_te = y_te.stack(['store_nbr', 'family'])
    y = pd.concat([y_tr, y_te])
    
    pred_train_y = pred_train_y.stack(['store_nbr', 'family'])
    pred_test_y = pred_test_y.stack(['store_nbr', 'family'])
    # If sales < 0 -> 0
    pred_y = pd.concat([pd.Series(pred_train_y).apply(lambda x: 0 if x<0 else x), pd.Series(pred_test_y).apply(lambda x: 0 if x<0 else x)])
    
    # Train
    if end_df <= date['date_end_train']:

        # Some plots to check 
        fig, axes = plt.subplots(4, 2, figsize=(15,10))
        y.loc['2017-02':,:].reset_index().groupby(by='date')[0].sum().plot(ax=axes[0,0], color="red")
        pred_y.loc['2017-02':,:].reset_index().groupby(by='date')[0].sum().plot(ax=axes[0,0], color="orange")
    
        y.loc['2017-08':,:].reset_index().groupby(by='date')[0].sum().plot(ax=axes[0,1], color="red")
        pred_y.loc['2017-08':,:].reset_index().groupby(by='date')[0].sum().plot(ax=axes[0,1], color="orange")
    
        y.loc['2017-02':,:].reset_index().query("family=='GROCERY I'").groupby(by='date')[0].sum().plot(ax=axes[1,0], color="red")
        pred_y.loc['2017-02':,:].reset_index().query("family=='GROCERY I'").groupby(by='date')[0].sum().plot(ax=axes[1,0], color="orange")
    
        y.loc['2017-02':,:].reset_index().query("family=='SCHOOL AND OFFICE SUPPLIES'").groupby(by='date')[0].sum().plot(ax=axes[1,1], color="red")
        pred_y.loc['2017-02':,:].reset_index().query("family=='SCHOOL AND OFFICE SUPPLIES'").groupby(by='date')[0].sum().plot(ax=axes[1,1], color="orange")
    
        (y.loc['2017-02':,:].reset_index().groupby(by='date')[0].sum() - pred_y.loc['2017-02':,:].reset_index().groupby(by='date')[0].sum()).plot(ax=axes[2,1])
        sns.residplot(ax=axes[2,0], x=pred_y.loc['2017-02':,:].reset_index().groupby(by='date')[0].sum(), y=zscore(y.loc['2017-02':,:].reset_index().groupby(by='date')[0].sum() - pred_y.loc['2017-02':,:].reset_index().groupby(by='date')[0].sum()), lowess=True, scatter_kws={'alpha': 0.5}, 
                     line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
        y.loc['2017-02':,:].reset_index().query("store_nbr==50").groupby(by='date')[0].sum().plot(ax=axes[3,0], color="red")
        pred_y.loc['2017-02':,:].reset_index().query("store_nbr==50").groupby(by='date')[0].sum().plot(ax=axes[3,0], color="orange")
    
        y.loc['2017-02':,:].reset_index().query("store_nbr==47").groupby(by='date')[0].sum().plot(ax=axes[3,1], color="red")
        pred_y.loc['2017-02':,:].reset_index().query("store_nbr==47").groupby(by='date')[0].sum().plot(ax=axes[3,1], color="orange")
        
        fig.tight_layout()
    # Test
    else:
        return pred_test_y, pred_y, y, df

In [ ]:
#model_func(final_df, date['date_end_train'], 60)

In [ ]:
#model_func(final_df, date['date_end_train'], 30)

In [ ]:
#model_func(final_df, date['date_end_train'], 16)

In [ ]:
#model_func(final_df, date['date_end_train'], 6)

In [ ]:
model_pred_test_y, model_pred_tot_y, orig_y, final_df = model_func(final_df, date['date_end_test'], 16)

In [ ]:
y = pd.Series(np.exp(model_pred_tot_y.values) - 1, index=model_pred_tot_y.index)

In [ ]:
sub = final_df.reset_index().set_index(['date', 'store_nbr', 'family']).sort_index().loc[date['date_start_fore']:, 'id']

In [ ]:
fin_sub = pd.concat([sub, y], axis=1)

In [ ]:
fin_sub = fin_sub.rename(columns={0:'sales'})

In [ ]:
fin_sub['sales'] = fin_sub['sales'].apply(lambda x: 0 if x<0 else x) 

In [ ]:
fin_sub = fin_sub.loc['2017-08-16':]

In [ ]:
fin_sub

In [ ]:
fin_sub.to_csv('submission.csv', index=False)